In [1]:
import findspark
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.conf import SparkConf
from pyspark.context import SparkContext as sc
from pyspark.sql.types import *
import psycopg2
import os
from pyspark.sql.functions import col,lit

findspark.init()
findspark.find()

'/home/jwang/workspace/national_energy_impact/.venv/lib/python3.8/site-packages/pyspark'

In [2]:
spark = SparkSession.builder \
        .appName("Python Spark SQL basic example") \
        .config('spark.jars', '/home/jwang/workspace/postgresql_jar/postgresql-42.6.0.jar') \
        .getOrCreate()

23/08/01 12:50:11 WARN Utils: Your hostname, jwang resolves to a loopback address: 127.0.1.1; using 192.168.2.168 instead (on interface wlp0s20f3)
23/08/01 12:50:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/08/01 12:50:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
def db_connect(host, database, user, password, port, schema, table_name):
    
    conn_string = 'postgresql://{0}:{1}@{2}:{3}/{4}'.format(user, password, host, port, database)
    pg_conn = psycopg2.connect(conn_string)
    cur = pg_conn.cursor()  
    cur.execute(f"CREATE SCHEMA IF NOT EXISTS {schema}")
    cur.execute(f"DROP TABLE IF EXISTS {schema}.{table_name}")
    pg_conn.commit()
    pg_conn.close()

def load_data_to_postgres(df, schema, table_name, host, database, user, password, port):
  
    dbtable = f"{schema}.{table_name}"
    url = "jdbc:postgresql://{0}:{1}/{2}".format(host, port, database)
    df.write.format("jdbc")\
        .option('driver', 'org.postgresql.Driver') \
        .option("url", url).option('dbtable', dbtable) \
        .option("user", user).option("password", password).save()
    print("Add to postgresql: " + dbtable)
    

In [4]:
def process_volume_data(file_name):
    try:
        file = open(file_name, 'r')
    except FileNotFoundError as e:
        # if no file found, return empty data frame
        sample_file_for_volume_schema = "../HPMS_Data_Unified_Name/HPMS_Data_Unified_Name/2022/jan_2022_ccs_data/AK_JAN_2022 (TMAS).VOL"
        df = spark.read.csv(sample_file_for_volume_schema, sep='|', header=True)
        columns = df.schema
        current_vol_df = spark.createDataFrame(data=[],schema=columns)
        return current_vol_df

    line = file.readline()
    file.close()
    if len(line) > 0:
        has_header = True
        if line[0:6] != 'record':
            has_header = False
        current_vol_df = spark.read.csv(file_name, sep='|', header=has_header)
    else: # if have no available data in the file, return an empty dataframe, eg. 'FL_NOV_2021 (TMAS).VOL'
        sample_file_for_volume_schema = "../HPMS_Data_Unified_Name/HPMS_Data_Unified_Name/2022/jan_2022_ccs_data/AK_JAN_2022 (TMAS).VOL"
        df = spark.read.csv(sample_file_for_volume_schema, sep='|', header=True)
        columns = df.schema
        current_vol_df = spark.createDataFrame(data=[],schema=columns)
    return current_vol_df
    
def process_volume_data_without_seperator(file_name, vol_data_size):
    sample_file_for_volume_schema = "../HPMS_Data_Unified_Name/HPMS_Data_Unified_Name/2022/jan_2022_ccs_data/AK_JAN_2022 (TMAS).VOL"
    df = spark.read.csv(sample_file_for_volume_schema, sep='|', header=True)
    columns = df.schema

    try:
        file = open(file_name, 'r')
    except FileNotFoundError as e:
        # if no file found, return empty data frame
        current_vol_df = spark.createDataFrame(data=[],schema=columns)
        return current_vol_df
    
    lines = file.readlines()
    all_data = []
    for line in lines:
        new_row = []
        idx = 0
        for station_data in vol_data_size:
            new_row.append(line[idx: idx+station_data[1]])
            idx += station_data[1]
        all_data.append(new_row)
    file.close()
    current_vol_df = spark.createDataFrame(data=all_data, schema=columns)
    return current_vol_df 

In [5]:
def main():
    states_dict = {
                'AK': 'Alaska',
                'AL': 'Alabama',
                'AR': 'Arkansas',
                'AZ': 'Arizona',
                'CA': 'California',
                'CO': 'Colorado',
                'CT': 'Connecticut',
                'DC': 'District of Columbia',
                'DE': 'Delaware',
                'FL': 'Florida',
                'GA': 'Georgia',
                'HI': 'Hawaii',
                'IA': 'Iowa',
                'ID': 'Idaho',
                'IL': 'Illinois',
                'IN': 'Indiana',
                'KS': 'Kansas',
                'KY': 'Kentucky',
                'LA': 'Louisiana',
                'MA': 'Massachusetts',
                'MD': 'Maryland',
                'ME': 'Maine',
                'MI': 'Michigan',
                'MN': 'Minnesota',
                'MO': 'Missouri',
                'MS': 'Mississippi',
                'MT': 'Montana',
                'NC': 'North Carolina',
                'ND': 'North Dakota',
                'NE': 'Nebraska',
                'NH': 'New Hampshire',
                'NJ': 'New Jersey',
                'NM': 'New Mexico',
                'NV': 'Nevada',
                'NY': 'New York',
                'OH': 'Ohio',
                'OK': 'Oklahoma',
                'OR': 'Oregon',
                'PA': 'Pennsylvania',
                'RI': 'Rhode Island',
                'SC': 'South Carolina',
                'SD': 'South Dakota',
                'TN': 'Tennessee',
                'TX': 'Texas',
                'UT': 'Utah',
                'VA': 'Virginia',
                'VT': 'Vermont',
                'WA': 'Washington',
                'WI': 'Wisconsin',
                'WV': 'West Virginia',
                'WY': 'Wyoming'
        }
    vol_data_size = (
        ('record_type',1),
        ('state_code',2),
        ('f_system',2),
        ('station_id',6),
        ('travel_dir',1),
        ('travel_lane',1),
        ('year_record',2),
        ('month_record',2),
        ('day_record',2),
        ('day_of_week',1),
        ('hour_00',5),
        ('hour_01',5),
        ('hour_02',5),
        ('hour_03',5),
        ('hour_04',5),
        ('hour_05',5),
        ('hour_06',5),
        ('hour_07',5),
        ('hour_08',5),
        ('hour_09',5),
        ('hour_10',5),
        ('hour_11',5),
        ('hour_12',5),
        ('hour_13',5),
        ('hour_14',5),
        ('hour_15',5),
        ('hour_16',5),
        ('hour_17',5),
        ('hour_18',5),
        ('hour_19',5),
        ('hour_20',5),
        ('hour_21',5),
        ('hour_22',5),
        ('hour_23',5),
        ('restrictions',1)
    )

    states = list(states_dict.keys())
    months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    schema_vol = 'volume'
    state_code = spark.read.csv('state_code.csv', header=True)
    host = 'localhost'
    database = 'national_volume_osm'
    user = 'postgres'
    password = 'wjh00813'
    port = 5432

    sample_file_for_volume_schema = "../HPMS_Data_Unified_Name/HPMS_Data_Unified_Name/2022/jan_2022_ccs_data/AK_JAN_2022 (TMAS).VOL"
    df = spark.read.csv(sample_file_for_volume_schema, sep='|', header=True)
    columns = df.schema
    years = [i for i in range(2022,2010,-1)]
    
    empty_tables = []

    for year in years:
        for state in states:
            vol_df = spark.createDataFrame(data=[], schema=columns)
            vol_file_names = ['../HPMS_Data_Unified_Name/HPMS_Data_Unified_Name/{0}/{1}_{0}_ccs_data/{2}_{3}_{0} (TMAS).VOL'.format(year, month, state, month.upper())\
                                for month in months]
            for file_name in vol_file_names:
                if year == 2021 or year == 2022:
                    current_vol_df = process_volume_data(file_name)
                else:
                    current_vol_df = process_volume_data_without_seperator(file_name, vol_data_size)
                vol_df = vol_df.union(current_vol_df)
            
            state_full_name = states_dict[state]
            code = state_code.filter(state_code.state == state_full_name).select('code').collect()[0][0]
            add_column = list(states_dict.keys())[list(states_dict.values()).index(state_full_name)]
            vol_df = vol_df.filter(vol_df.state_code == code).withColumn('state_name', lit(add_column))
            table_name = f"{add_column}_{year}"
            db_connect(host, database, user, password, port, schema_vol, table_name)
            load_data_to_postgres(vol_df, schema_vol, table_name, host, database, user, password, port)
            if vol_df.count() < 1:
                empty_tables.append(table_name)

    print("Empty volume tables are: ", empty_tables)

In [6]:
def test_read_from_postgresql():
    # only to test reading from postgresql
    host = 'localhost'
    database = 'national_volume_osm'
    user = 'postgres'
    password = 'wjh00813'
    port = 5432
    url = "jdbc:postgresql://{0}:{1}/{2}".format(host, port, database)

    df = spark.read.format("jdbc")\
        .option('driver', 'org.postgresql.Driver') \
        .option("url", url).option('dbtable', 'volume.ak_2022') \
        .option("user", user).option("password", password).load()

    print(len(df.schema))
    print(df.count())
    df.show()

In [7]:
if __name__ == "__main__":
    main()

23/08/01 12:50:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Add to postgresql: volume.AK_2022


Add to postgresql: volume.AL_2022
Add to postgresql: volume.AR_2022


Add to postgresql: volume.AZ_2022


Add to postgresql: volume.CA_2022
Add to postgresql: volume.CO_2022
Add to postgresql: volume.CT_2022
Add to postgresql: volume.DC_2022
Add to postgresql: volume.DE_2022


...

Add to postgresql: volume.VT_2011


Add to postgresql: volume.WA_2011


Add to postgresql: volume.WI_2011


Add to postgresql: volume.WV_2011


Add to postgresql: volume.WY_2011
Empty volume tables are:  []
